# Keyword Extraction by TF-IDF 

In [1]:
import pandas as pd
import re
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import json
with open('./review.json', 'r') as f:
    review = json.load(f)


In [3]:
df = pd.DataFrame(review)

In [7]:
df.dropna(subset=['visit_mean'], inplace=True)

In [10]:
df.reset_index(drop=True, inplace=True)
df

,MCT_NM,review,visit_mean
0,탕탕이2호점,"[맛있어요, 맛있어요, 비싸지만 아주 맛있게 먹었습니다!!!, 재료가 신선하고 푸짐...",1.047619
1,뚱돼지,"[이런글 쓰고 싶진 않은데, 죄송하지만 살면서 먹은 고기중에 제일 별로였습니다......",1.064516
2,송이축산정육식당,[점심시간 메뉴를 고민하다가 최근 신메뉴가 나왔다고해서 먹으러 왔습니다. 원래 돼기...,1.960000
3,24시뼈다귀탕 신서귀점,[오랫만에 다시 찾은 곳💗\n여전히 맛있고 친절하시고~\n술맛도 너무 좋아요...!...,1.633333
4,명당양과노형점,"[옛 기억의 동네 빵집, 동네 빵집이에요, 맛있어요, 좋아요, 맛있어요, 좋아요, ...",2.680000
...,...,...,...
165,큰가름본점,"[ㅇㅇ, 감자탕 야채가 좀 부족한듯했어요., 맛있어서 가는곳인데 새해들어 볶음밥가격...",1.250000
166,뚜레쥬르 제주아라아이파크점,"[직원분이 친절하셨어요 빵 잘 구입했습니다, 아르바이트 하시는 직원분이 정말 친절하...",2.580000
167,마라힐링,[3번째방문인데 리뷰는 첨쓰네용\n넘맛있어요ㅎ\n꿔바로우소스는많이셔서 주의해야하지만...,1.407407
168,곽지해녀의집,[1년전인가 2년전인가 우연히 방문했다가 그동안 먹어봤던 물회와는 전혀다른 담백깔끔...,1.020000


In [11]:
okt = Okt()

def remove_non_korean(text):
    """리뷰에서 한글을 제외한 모든 문자 제거"""
    return re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\s]", "", text)


def load_stopwords(filepath="k_stopword.txt"):
    """불용어 txt 파일을 읽어 불용어 리스트 반환"""
    with open(filepath, "r", encoding="utf-8") as f:
        stopwords = f.read().splitlines()
    return stopwords

stopwords = load_stopwords()


def okt_tokenize(text):
    """Okt를 이용한 형태소 분석 및 불필요 품사 제거"""
    # 형태소 분석 및 품사 태깅
    words = okt.pos(text, stem=True)  # 어간 추출
    
    # 필요한 품사만 필터링 (명사, 동사, 형용사, 부사)
    selected_words = [
        word for word, pos in words if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']
    ]
    
    # 불용어 제거
    filtered_words = [word for word in selected_words if word not in stopwords]
    
    # 최종 단어들을 문자열로 반환
    return " ".join(filtered_words)


def preprocess_review(text):
    """리뷰 텍스트를 전처리하고 형태소 분석한 최종 결과 반환"""
    text = remove_non_korean(text)
    processed_text = okt_tokenize(text)
    
    return processed_text

def preprocess_review_list(text_list):
    """리뷰 텍스트를 전처리하고 형태소 분석한 최종 결과 반환"""
    preprocessed_text_list = []
    for text in text_list:
        text = remove_non_korean(text)
        processed_text = okt_tokenize(text)
        preprocessed_text_list.append(processed_text) 

    #text = remove_non_korean(text)
    #processed_text = okt_tokenize(text)

    preprocessed_text = " ".join(preprocessed_text_list)
    
    return preprocessed_text

In [12]:
# TF-IDF 키워드 추출 함수
def extract_keywords_tfidf(corpus, top_n):
    """TF-IDF를 사용해 각 문서의 상위 N개 키워드 추출"""
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    features = vectorizer.get_feature_names_out()
    
    keywords = []
    for i in tqdm(range(X.shape[0])):
        row = X[i].toarray().flatten()
        top_indices = row.argsort()[-top_n:][::-1]
        top_keywords = [features[idx] for idx in top_indices]
        keywords.append(top_keywords)
    return keywords

In [14]:
df['pos_tag'] = df['review'].apply(preprocess_review_list)

In [39]:
# 이모티콘만 있는 경우 pos tag 값이 비어있으므로 제거

df = df[df['pos_tag'].apply(lambda x: len(x)>=10)]
df.reset_index(drop=True, inplace=True)

In [40]:
# pos_tag 열에서 키워드 추출
df['tf_idf_keywords'] = extract_keywords_tfidf(df['pos_tag'].tolist(),20)

100%|██████████| 169/169 [00:00<00:00, 8400.84it/s]
/var/folders/y3/l8bvkgx53cd0nbrk2b450t7m0000gn/T/ipykernel_10937/967659636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tf_idf_keywords'] = extract_keywords_tfidf(df['pos_tag'].tolist(),20)


In [ ]:
#!pip install krwordrank

In [18]:
from krwordrank.word import KRWordRank

def krword(texts):
    #for texts in text_list:
    min_count = 1   # 단어의 최소 출현 빈도수 (그래프 생성 시)
    max_length = 10 # 단어의 최대 길이
    wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)
    beta = 0.85    # PageRank의 decaying factor beta
    max_iter = 20
    t_list = []
    for text in texts:
          kr_text = remove_non_korean(text)
          processed_text = okt_tokenize(kr_text)
          t_list.append(processed_text)
    #texts = split_noun_sentences(text)
    keywords, rank, graph = wordrank_extractor.extract(t_list, beta, max_iter)
    output = []
    num = 0
    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True):
            #print('%8s:\\t%.4f' % (word, r))
            output.append(word)
            num += 1
            if num == 21:
                  break
    
    return output


In [41]:
df['krword_rank'] = df['review'].apply(krword)

  4%|▍         | 20/503 [00:00<00:00, 665762.54it/s]
/var/folders/y3/l8bvkgx53cd0nbrk2b450t7m0000gn/T/ipykernel_10937/632893083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['krword_rank'] = df['review'].apply(krword)


In [42]:
df

,MCT_NM,review,visit_mean,pos_tag,tf_idf_keywords,krword_rank
0,탕탕이2호점,"[맛있어요, 맛있어요, 비싸지만 아주 맛있게 먹었습니다!!!, 재료가 신선하고 푸짐...",1.047619,맛있다 맛있다 비싸다 아주 맛있다 먹다 재료 신선하다 푸다 짐해 시청 점 먹다 호점...,"[육회, 맛있다, 호점, 쏘다, 먹다, 신선하다, 좋다, 산낙지, 스굿, 불토, 연...","[맛있, 먹다, 친절하다, 신선하다, 육회, 밑반찬, 계란, 추천, 좋다, 쏘다, ..."
1,뚱돼지,"[이런글 쓰고 싶진 않은데, 죄송하지만 살면서 먹은 고기중에 제일 별로였습니다......",1.064516,글 쓰다 싶다 죄송하다 살다 먹다 고기 제일 별로 밑반찬 맛 평범하다 고깃집 고기 ...,"[고기, 맛있다, 양념, 돼지갈비, 갈비, 뚱돼, 겹살, 친절하다, 좋다, 먹다, ...","[맛있다, 고기, 친절, 가다, 먹다, 양념, 너무, 진짜, 직원, 사장, 굽다, ..."
2,송이축산정육식당,[점심시간 메뉴를 고민하다가 최근 신메뉴가 나왔다고해서 먹으러 왔습니다. 원래 돼기...,1.960000,점심시간 메뉴 고민 최근 메뉴 나오다 먹다 오다 원래 돼다 고기 가성 비 갑인 식당...,"[고기, 굳다, 대패, 맛있다, 축산, 송이, 정육, 좋다, 찌개, 먹다, 가성, ...","[맛있다, 고기, 좋다, 먹다, 정말, 친절, 맛집, 메뉴, 직원, 자다, 반찬, ..."
3,24시뼈다귀탕 신서귀점,[오랫만에 다시 찾은 곳💗\n여전히 맛있고 친절하시고~\n술맛도 너무 좋아요...!...,1.633333,만 다시 찾다 곳 여전하다 맛있다 친절하다 술맛 너무 좋다 수제비 볶음밥 필수 뭐 ...,"[감자탕, 맛있다, 볶음밥, 수제비, 먹다, 수도권, 미나리, 좋다, 식이섬유, 식...","[맛있다, 먹다, 감자탕, 좋다, 볶음밥, 식사, 너무, 국물, 가다, 수제비, 진..."
4,명당양과노형점,"[옛 기억의 동네 빵집, 동네 빵집이에요, 맛있어요, 좋아요, 맛있어요, 좋아요, ...",2.680000,옛 기억 동네 빵집 동네 빵집 맛있다 좋다 맛있다 좋다 사장 엄청 친절하다 감동 받...,"[빵집, 맛있다, 좋다, 쌀빵, 카레빵, 보로, 방앗간, 앗다, 루베, 도넛, 체적...","[맛있, 늦다, 빵집, 사장, 친절하다, 어렵다, 응대, 많다, 많이, 마다, 느낌..."
...,...,...,...,...,...,...
164,큰가름본점,"[ㅇㅇ, 감자탕 야채가 좀 부족한듯했어요., 맛있어서 가는곳인데 새해들어 볶음밥가격...",1.250000,감자탕 야채 부족하다 맛있다 가다 곳 새해 들다 볶음밥 가격 원 너무 비싸다 가다...,"[감자탕, 해물, 볶음밥, 맛있다, 먹다, 샤브샤브, 태도, 국물, 진짜, 친절하다...","[먹다, 감자탕, 맛있다, 해물, 진짜, 고기, 생각, 친절하다, 국물, 맛집, 가..."
165,뚜레쥬르 제주아라아이파크점,"[직원분이 친절하셨어요 빵 잘 구입했습니다, 아르바이트 하시는 직원분이 정말 친절하...",2.580000,직원 친절하다 빵 자다 구입 아르바이트 직원 정말 친절하다 가다 정말 불친절하다 사...,"[식빵, 아르바이트, 아라동, 친절하다, 맛있다, 결재, 뚜레쥬르, 먹다, 초코, ...","[맛있다, 친절하다, 가다, 먹다, 좋다, 직원, 할인, 너무, 아라동, 정말, 쓰..."
166,마라힐링,[3번째방문인데 리뷰는 첨쓰네용\n넘맛있어요ㅎ\n꿔바로우소스는많이셔서 주의해야하지만...,1.407407,번 방문 리뷰 첨 쓰다 용 넘다 맛있다 꾸다 바 로우 소스 셔 주의 맛있다 새우 볶...,"[마르다, 단계, 맛있다, 꾸다, 먹다, 탕집, 볶음밥, 위생, 좋다, 이모, 도전...","[마르, 먹다, 맛있다, 좋다, 재료, 사장, 도전, 음식, 단계, 위생, 볶음밥,..."
167,곽지해녀의집,[1년전인가 2년전인가 우연히 방문했다가 그동안 먹어봤던 물회와는 전혀다른 담백깔끔...,1.020000,전인 전인 우연히 방문 그동안 먹다 보다 물회 전혀 담 깔끔하다 맛 반하다 재 방문...,"[먹다, 맛있다, 해산물, 좋다, 어회, 고등, 신선하다, 바다, 노을, 전복죽, ...","[맛있다, 먹다, 좋다, 너무, 신선하다, 오다, 해산물, 엄청, 완전, 보다, 나..."


# GEMINI

In [46]:
import google.generativeai as genai
import time
api_key = ###
genai.configure(api_key = api_key)
mct_keyword = {}
llm = genai.GenerativeModel("gemini-1.5-flash")

num = 1
for place, pos_tag in tqdm(df[['MCT_NM','pos_tag']].values):

        context = pos_tag
    
                        
        question = """
        MBTI 별 맛집 성향에 따른 음식점 추천을 하려고 해. 주어진 제주도 맛집에 대한 리뷰 명사 및 동사 텍스트 기반해 주요 20개의 키워드를 추출해줘. 설명 없이 키워드만 나열해줘.
        키워드를 통해 맛, 만족도, 서비스, 분위기, 음식량 등을 파악할 수 있도록 추출해줘.

        """

        messages = f"""
            주어진 리뷰 명사 및 동사 텍스트를 기반으로 사용자의 질문에 답변하세요.
            답변은 한국어로 작성하세요.

            리뷰 명사 및 동사 텍스트: {context}

            사용자 질문: {question}
            
            답변 예시 : 음식점1 : [키워드1, 키워드2, 키워드3, 키워드4, 키워드5, 키워드6, 키워드7, 키워드8, 키워드9, 키워드10, 키워드11, 키워드12, 키워드13, 키워드14, 키워드15, 키워드16, 키워드17, 키워드18, 키워드19, 키워드20]

            """

        try:
                response = llm.generate_content(
                                messages,
                                #safety_settings="BLOCK_NONE",
                        
                        )
                time.sleep(3)
                mct_keyword[place] = response.text
                num += 1
        except Exception as e:
                time.sleep(5)
                print(e)
                num += 1
                llm = genai.GenerativeModel("gemini-1.5-flash")
                response = llm.generate_content(
                                messages,
                                #safety_settings="BLOCK_NONE",
                        
                        )
                if response.text != '':
                        mct_keyword[place] = response.text
                        num += 1
                else:
                        pass

        if num % 10 == 0:
                # print(f'{num}번째 키워드 추출 완료')
                time.sleep(3)
                llm = genai.GenerativeModel("gemini-1.5-flash")


/opt/homebrew/Caskroom/miniforge/base/envs/bigcon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  5%|▍         | 8/169 [00:30<10:06,  3.76s/it]

10번째 키워드 추출 완료


 11%|█         | 18/169 [01:12<09:42,  3.86s/it]

20번째 키워드 추출 완료


 17%|█▋        | 28/169 [01:53<08:59,  3.83s/it]

...

 92%|█████████▏| 156/169 [11:18<00:50,  3.89s/it]

160번째 키워드 추출 완료


 98%|█████████▊| 166/169 [12:02<00:11,  3.89s/it]

170번째 키워드 추출 완료


100%|██████████| 169/169 [12:16<00:00,  4.36s/it]


In [47]:
mct_keyword

{'탕탕이2호점': '[맛있다, 신선하다, 푸짐하다, 양도 많다, 배부르다, 친절하다, 좋다, 밑반찬, 솜씨, 서비스, 굿, 추천, 단골, 육회, 낙지, 연포탕, 비빔밥, 계란말이, 맥주, 불토] \n',
 '뚱돼지': '[흑돼지, 돼지갈비, 양념갈비, 맛있다, 친절하다, 깔끔하다, 맛집, 최고, 굳, 찌다, 존맛, 추천, 서비스, 싱싱, 육즙, 푸짐, 만족, 기분 좋다, 좋다, 강추] \n',
 '송이축산정육식당': '[가성비, 맛있다, 맛집, 돼지고기, 신선하다, 친절하다, 좋다, 굳, 푸짐하다, 착하다, 맛, 만족, 육즙, 부드럽다, 깔끔하다, 넉넉하다, 싱싱하다, 푸짐,  저렴하다,  양] \n',
 '24시뼈다귀탕 신서귀점': '[맛있다, 푸짐하다, 친절하다, 든든하다, 술안주, 볶음밥, 수제비, 감자탕, 뼈, 국물, 시원하다, 매운탕, 콩나물, 미나리, 우거지, 야채, 칼칼하다, 깔끔하다, 넉넉하다, 맛집] \n',
 '명당양과노형점': '[맛있다, 좋다, 친절하다, 감동, 비싸다, 다이어트, 생일 케잌, 굿, 깔끔하다, 달달, 무난, 친절, 생각나다, 인기 많다, 단체 주문, 최고, 많다, 팥소, 늦다, 쌀 카레빵] \n',
 '바로족발보쌈': '[족발, 맛있다, 부드럽다, 쫄깃, 돼지, 냄새, 없다, 막국수, 순두부찌개, 푸짐, 양, 많다, 친절, 서비스, 웨이팅, 줄, 서다, 기다리다, 손님, 많다] \n',
 '송쿠쉐': '송쿠쉐 : 스테이크, 맛있다, 분위기, 고급스럽다, 친절하다, 런치, 코스, 가성비, 데이트, 기념일,  예약,  세트,  와인,  음식,  양,  재료,  신선하다,  특별하다,  만족,  강추 \n',
 '남녕국수': '[집밥, 반찬, 맛있다, 친절하다, 먹다, 가다, 오다, 이전, 맛, 가격, 착하다, 정말, 맛집, 너무, 진정하다, 좋다, 만,  먹다,  가다,  오다] \n',
 '고촌': '[갈매기, 맛있다, 좋다, 부드럽다, 술, 싸다, 친절하다, 구이, 먹다, 괜찮다, 갈매기살, 반하다, 사장, 부심, 욤, 